In [1]:
import torch
import numpy as np
import pandas as pd
import codecs
import re
import nltk

from nltk.stem import WordNetLemmatizer

from random import shuffle

from collections import Counter

from numpy import array

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

from torch import tensor
from torch import nn
from torch import optim
from torch.autograd import Variable

# Parameters

In [2]:
# Number of recognized words you put in input
nb_input = 100 #8414 max

# Number of classe, constant
nb_output = 6

# Number of hidden layers
nb_hidd_lay = 1
hidden_size = 1

# Learning rate
lr = 0.005

# Number of epochs
nb_epochs = 5

use_cuda = torch.cuda.is_available()

### 0.Preprocessing

In [3]:
# Encoding in windows-1252, utf-8 generate error on some char
file = codecs.open("train_all.label", "r+","windows-1252")
train_data = []
for line in file.readlines():
    train_data.append(line)

# Now test set
file = codecs.open("TREC_test.label", "r+","windows-1252")
test_data = []
for line in file.readlines():
    test_data.append(line)

In [4]:
# Lemmatisation
# Working on list_of_words
# TODO: it doesn't work
def lemm(phrase):
    lemmer = WordNetLemmatizer()
    lemmed = []
    for word in phrase:
        lemmed.append(lemmer.lemmatize(word, pos="v"))
    return lemmed

In [5]:
# Divided file into 2 list:
# questions = list of questions 
# labels = list of labels

questions = []
labels = []

# Black list
regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')

(train_data[0]).split()[0]
for string in train_data:
    question_str = []
    for x in lemm(string.split()[1:]):
        question_str.append(x.lower())
    labels.append(string.split()[0])
    questions.append(question_str)
print("len(questions) = " + str(len(questions)))
print("Exemples of questions:")
print(questions[:2])



questions_test = []
labels_test = []

(test_data[0]).split()[0]
for string in test_data:
    question_str = []
    for x in lemm(string.split()[1:]):
        question_str.append(x.lower())
    labels_test.append(string.split()[0])
    questions_test.append(question_str)
print()
print("And in the test set:")
print("len(questions_test) = " + str(len(questions_test)))
print("Exemples of questions:")
print(questions_test[:2])



len(questions) = 15452
Exemples of questions:
[['how', 'do', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'russia', '?'], ['what', 'film', 'feature', 'the', 'character', 'popeye', 'doyle', '?']]

And in the test set:
len(questions_test) = 500
Exemples of questions:
[['how', 'far', 'be', 'it', 'from', 'denver', 'to', 'aspen', '?'], ['what', 'county', 'be', 'modesto', ',', 'california', 'in', '?']]


In [6]:
# Method to add tags begin and end to phrases list.
# /!\ 
# WARNING : this method need to be executed only ONE time.
# /!\
def add_tag(question_list):
    for i in range(0, len(question_list)):
        if question_list[i][0] != '<bos>' :
            question_list[i].insert(0, '<bos>')
            question_list[i].append('<eos>')
add_tag(questions)
add_tag(questions_test)

In [7]:
# questions

In [8]:
%%time
# Vocabulary of unique words
data = []
data.append('<unk>')
for q in questions:
    for word in q:
        data.append(word)
print(len(data))

188317
CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 27.9 ms


In [9]:
scv = np.array(data)
unik, counts = np.unique(scv,return_counts=True)
vocabulary = {}
for i in range(0, len(unik)):
    vocabulary[unik[i]] = counts[i]

In [10]:
vocabulary

{'falls': 7,
 'territory': 9,
 'database': 9,
 'chickenpoxs': 3,
 'nearest': 3,
 'cinzano': 3,
 'eidologist': 1,
 'cholera': 3,
 'registration': 2,
 'switch': 2,
 'smothers': 1,
 'users': 2,
 'treatments': 2,
 'taj': 3,
 'nanometer': 5,
 'fig': 3,
 'developmental': 5,
 'shots': 5,
 'yahoo': 12,
 'god-sullen': 5,
 'political': 6,
 'cure': 1,
 'depend': 4,
 'hollywood': 18,
 'hall': 8,
 'accommodate': 1,
 'mammal': 12,
 'wisconsin': 4,
 'cologne': 4,
 'sugar': 5,
 'highest': 87,
 'multicultural': 1,
 'rockin': 1,
 'stat': 4,
 'anka': 4,
 'lou': 8,
 'volleyball': 1,
 'insure': 5,
 'maya': 2,
 'viagra': 4,
 'inhabit': 5,
 'definition': 44,
 'al-farri': 5,
 'determinism': 2,
 'spray': 8,
 'chairbound': 5,
 'grade': 2,
 'sensitive': 12,
 'travels': 6,
 'wonders': 7,
 'active': 8,
 'pitchers': 4,
 'zodiacal': 7,
 'slinky': 4,
 'formula': 6,
 'promoters': 5,
 'must': 18,
 'ash': 6,
 '9971': 3,
 'basic': 1,
 'airforce': 1,
 'leg': 9,
 'christopher': 9,
 'rom': 1,
 'theatrical': 1,
 'long': 167,

In [11]:
# Less commons words in your vocabulary
Counter(vocabulary).most_common(nb_input-1)[-10:]

[('become', 142),
 ('day', 141),
 ('die', 140),
 ('stand', 140),
 ('tv', 137),
 ('your', 137),
 ('or', 137),
 ('man', 136),
 ('show', 131),
 ('one', 130)]

In [12]:
word_list = list([x[0] for x in Counter(vocabulary).most_common(nb_input-1)])

# We add the unk word for future purpose.
word_list.append('<unk>')
words_array = np.array(word_list)
print("Vocabulary contains", len(words_array), "words.")

Vocabulary contains 100 words.


In [13]:
# Integer encoding with OneHotEncoder
words_tre = words_array.reshape(len(words_array),1)
one_hot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = one_hot_encoder.fit_transform(words_tre)
print(onehot_encoded)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [14]:
# Creating a dictionnary of word and its one hot array
words_onehoted = {}
for i in range(0, len(words_array)):
    words_onehoted[word_list[i]] = onehot_encoded[i]

In [15]:
# Function to get the corresponding one hot list for a word.
def get_onehot_word(word):
    if word in words_onehoted:
        return list(words_onehoted[word])
    else:
        return list(words_onehoted['<unk>'])

In [16]:
oh = get_onehot_word('<unk>')
one = oh.index(1.0)
one

8

In [17]:
# Testing if an unknown word is transformed into a <unk>
get_onehot_word('obviously_an_unknown_word').index(1.0)

8

In [18]:
# One hot categories

categories_onehoted = {}
categories_onehoted['ABBR'] = [1, 0, 0, 0, 0, 0] # Abbreviation
categories_onehoted['ENTY'] = [0, 1, 0, 0, 0, 0] # Entity
categories_onehoted['DESC'] = [0, 0, 1, 0, 0, 0] # Description
categories_onehoted['HUM']  = [0, 0, 0, 1, 0, 0] # Human
categories_onehoted['LOC']  = [0, 0, 0, 0, 1, 0] # Location
categories_onehoted['NUM']  = [0, 0, 0, 0, 0, 1] # Numeric

In [19]:
# Function to get the corresponding one hot list for a category.
def get_onehot_category(category):
    return categories_onehoted[category]

In [20]:
oh = get_onehot_category('HUM')
one = oh.index(1.0)
one

3

In [21]:
%%time
# Creating training set

batch_data = []
for num_question in range(len(questions)):
    # Construction of question_onehot list.
    question_onehot = [get_onehot_word(word) for word in questions[num_question]]
    
    # Construction of category_onehot.
    category = labels[num_question].partition(':')[0]
    category_onehot = get_onehot_category(category)
    batch_data.append([tensor([question_onehot]), tensor([category_onehot])])
    

CPU times: user 2.22 s, sys: 108 ms, total: 2.33 s
Wall time: 2.28 s


In [22]:
%%time
# Creating test set

batch_data_test = []
for num_question in range(len(questions_test)):
    
    # Construction of question_onehot list.
    question_onehot = [get_onehot_word(word) for word in questions[num_question]]
    
    # Construction of category_onehot.
    category = labels_test[num_question].partition(':')[0]
    category_onehot = get_onehot_category(category)
    batch_data_test.append([tensor([question_onehot]), tensor([category_onehot])])
    

CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 64.7 ms


# RNN implementation
Using ReLU, and CrossEntropy

In [23]:
class RNN(nn.Module):
    def __init__(self, nb_inputs, nb_layers, nb_neurons, nb_outputs, learning_rate):
        super(RNN, self).__init__()
        
        # Applying RNN layer, and softmax then
        self.rnn = nn.RNN(input_size=nb_inputs, num_layers=nb_layers,
                   hidden_size=nb_neurons, dropout=0.5, batch_first=True, nonlinearity='relu')
        self.inter = nn.Linear(nb_neurons, nb_outputs)
        self.sm = nn.Softmax(dim=2)
        
        # Other usefull variables here
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.input_dim = nb_inputs
        self.output_dim = nb_output
        self.nb_layers = nb_layers
        self.nb_neurons = nb_neurons
        #self.synapses = Variable(torch.zeros(self.nb_layers, 1, self.nb_neurons))
        
    def forward(self, inputs):
        h0 = Variable(torch.zeros(self.nb_layers, inputs.size(0), self.nb_neurons))
        if use_cuda:
            h0 = h0.to("cuda")
        
        x, hn = self.rnn(inputs, h0)
        
        x = self.inter(x)
        x = nn.functional.softmax(x, dim=2)
        return x

# End of the class RNN

# Now let's define learn(), which learn a RNN some data
def learn(rnn, batch_list, num_epochs=1):
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        rnn.cuda(device)
    
    # Preparing
    rnn.train()
    losses = []
    criterion = nn.CrossEntropyLoss()
    
    # Shuffling batch_list
    shuffle(batch_list)

    for epoch in range(num_epochs):
        for batch_idx, (data, target) in enumerate(batch_list):
            
            data, target = data.to(device), target.to(device)
            
            #data, target = Variable(data), Variable(target)
            
            output = rnn(data)
            loss = criterion(output, target)
            
            losses.append(loss.data.item())

            rnn.optimizer.zero_grad()

            loss.backward()
            rnn.optimizer.step()
            
            # Print the progress
            if batch_idx % 100 == 0 or batch_idx % 100 == 1 or batch_idx == len(batch_list)-1:
                print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                        epoch, 
                        (batch_idx+1) * len(data), 
                        len(batch_list),
                        100. * (batch_idx+1) / len(batch_list), 
                        loss.data.item()), 
                        end='')
        print()
        
    # Return losses list, you can print them later if you want
    return losses


# return (rightAnswer, ignored, falseAnswer)
def getEfficience(rnn, batch_list, tresh=0) :
    rightAnswer = 0
    ignored = 0
    falseAnswer = 0
    
    device = torch.device("cuda" if use_cuda else "cpu")
    for (data, target) in batch_list :
        data, target = data.to(device), target.to(device)
        predicted = rnn(data).detach().cpu().numpy()[-1][-1]
        #print("predicted: "+str(np.argmax(predicted)))
        #print("target: "+str(np.argmax(target.detach().cpu().numpy()[-1])))
        if max(predicted) < tresh :
            ignored += 1
        else:
            if np.argmax(predicted) == np.argmax(target.detach().cpu().numpy()[-1]):
                rightAnswer += 1
            else:
                falseAnswer += 1
    return (rightAnswer, ignored, falseAnswer)

# Using the RNN

In [24]:
rnn = RNN(nb_inputs = nb_input, nb_layers=nb_hidd_lay,
          nb_neurons=hidden_size, nb_outputs=nb_output, learning_rate=lr)
if use_cuda:
    rnn = rnn.to("cuda")
losses = learn(rnn, batch_data, nb_epochs)
print("Done :)")

/info/etu/m2/i140302/venv/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


 Train Epoch: 0 [15452/15452 (100%)]	 Loss: 2.213928
 Train Epoch: 1 [15452/15452 (100%)]	 Loss: 2.214175
 Train Epoch: 2 [15452/15452 (100%)]	 Loss: 2.214204
 Train Epoch: 3 [15452/15452 (100%)]	 Loss: 2.214206
 Train Epoch: 4 [15452/15452 (100%)]	 Loss: 2.214208
Done :)


## Error curve

In [25]:
from scipy.signal import savgol_filter
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

x = np.linspace(0, len(losses), len(losses))
fig = plt.figure(figsize=(13, 8)) 
ax = fig.add_subplot(1,1,1)
cnn_line, = ax.plot(x, losses)

def update_losses(smooth=51):
    cnn_line.set_ydata(savgol_filter(losses, smooth, 3))
    fig.canvas.draw()

interact(update_losses, smooth=(5, 201, 2));

interactive(children=(IntSlider(value=51, description='smooth', max=201, min=5, step=2), Output()), _dom_class…

# Analysis on test set

In [26]:


final_results = getEfficience(rnn, batch_data)
total = sum(final_results)

correct = final_results[0]/total*100
ignored = final_results[1]/total*100
false = final_results[2]/total*100



print("Congratulations! On the training set:")
print("Corrects: " + str(correct) + "%")
#print("Ignored:  " + str(ignored) + "%")
print("False:    " + str(false) + "%")
print()

final_results = getEfficience(rnn, batch_data_test)
total = sum(final_results)

correct = final_results[0]/total*100
ignored = final_results[1]/total*100
false = final_results[2]/total*100



print("Congratulations! On the test set:")
print("Corrects: " + str(correct) + "%")
#print("Ignored:  " + str(ignored) + "%")
print("False:    " + str(false) + "%")

print()
print("A présent, tu peux copier-coller ça dans le doc sur le drive :)")
print(str(nb_input)+"\t"+str(lr)+"\t"+str(nb_epochs)+"\t"+str(nb_hidd_lay)+"\t"+str(hidden_size)+"\t\t"+str(correct)+"%")
print()



Congratulations! On the training set:
Corrects: 23.472689619466735%
False:    76.52731038053327%

Congratulations! On the test set:
Corrects: 18.8%
False:    81.2%

A présent, tu peux copier-coller ça dans le doc sur le drive :)
100	0.005	5	1	1		18.8%

